In [4]:
%pip install --quiet langchain-teddynote bs4

Note: you may need to restart the kernel to use updated packages.


In [22]:
import bs4
from langchain import hub
from langchain_community.llms import GPT4All
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StreamingStdOutCallbackHandler

In [8]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [16]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG


In [17]:
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)

In [64]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.

# 뉴스 데이터를 배열로 받는다고 가정합니다.
news_data = [
    "https://n.news.naver.com/mnews/article/003/0012813373",
    "https://n.news.naver.com/mnews/article/050/0000080411",
    # 추가적인 URL을 여기에 추가하세요
]

# 결과를 저장할 빈 리스트를 생성합니다.
all_splits = []

# news_data에 있는 URL을 반복적으로 처리합니다.
for news_url in news_data:
    # 각 뉴스 URL에 대해 WebBaseLoader를 설정합니다.
    loader = WebBaseLoader(
        web_paths=(news_url,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                "div",
                attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
            )
        ),
    )
    
    # 문서 로드
    docs = loader.load()
    print(f"URL: {news_url} - 문서의 수: {len(docs)}")
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    all_splits.extend(splits)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()


URL: https://n.news.naver.com/mnews/article/003/0012813373 - 문서의 수: 1
URL: https://n.news.naver.com/mnews/article/050/0000080411 - 문서의 수: 1


In [76]:
from langchain_core.prompts import PromptTemplate

# prompt = PromptTemplate.from_template(
#     """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
# 검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
# 한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

# #Question: 
# {question} 

# #Context: 
# {context} 

# #Answer:"""
# )

prompt = PromptTemplate.from_template(
    """당신은 최신 금융 동향을 분석하고 제공하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)을 바탕으로 금융 시장에 대한 동향을 전달하는 것입니다.
검색된 다음 문맥(context)을 사용하여 최신 금융 시장 동향을 요약해 주세요. 만약, 주어진 문맥(context)에서 동향을 찾을 수 없다면, `주어진 정보에서 금융 시장 동향에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#문맥(Context): 
{context} 

#금융 시장 동향 요약:"""
)

In [72]:
EEVE_Korean_Instruct = "C:/Users/David/AppData/Local/nomic.ai/GPT4All/EEVE-Korean-Instruct-10.8B-v1.0-Q4_0.gguf"  # 원하는 로컬 파일 경로로 대체하세요
gpt4all_falcon = "C:/Users/David/AppData/Local/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf"
llama3_8b ="C:/Users/David/AppData/Local/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf"


In [91]:
llm = GPT4All(
    model=gpt4all_falcon,
    # backend="gpu",  # GPU 설정
    streaming=True,  # 스트리밍 설정
    callbacks=[StreamingStdOutCallbackHandler()],  # 콜백 설정
)

# 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [92]:
from langchain_teddynote.messages import stream_response

In [93]:
answer = rag_chain.stream("금융 시장 동향을 요약해주세요.")
stream_response(answer)

ERROR: The prompt size exceeds the context window size and cannot be processed.ERROR: The prompt size exceeds the context window size and cannot be processed.

In [95]:
response = llm("한국어로 답변할 수 있습니다?")

response


Sure, here's a sample answer to the question "What is your favorite Korean dish?" in Korean:

밥은 이야기로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아

'\nSure, here\'s a sample answer to the question "What is your favorite Korean dish?" in Korean:\n\n밥은 이야기로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아요하면 지발지로 맛집하면 좋아'